# Problem Statement:

**How accurately can the words in a subreddit's post predict whether the post was intended for a 'good advice' or 'bad advice' subreddit?**

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import requests
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'regex'

# API & JSON:

In [1]:
def generate_json_posts(subreddit_str, size):
    
    # Setup URL of API
    base_url = "https://api.pushshift.io/reddit/search/submission"    
    
    # Create the params of the API URL
    params = {
        "subreddit": subreddit_str,
        "size": size
    }

    # Response
    res = requests.get(base_url, params)
    res_check = res.status_code
    
    # Check response is good
    if (res_check >= 200 and res_check < 300):
        
        # Create JSON:
        data = res.json()
        posts = data["data"]
        
        return posts
    else:
        return f"Check HTTP Error: {res_check}"

In [2]:
lpt_posts = generate_json_posts("LifeProTips", 1000)
ulpt_posts = generate_json_posts("UnethicalLifeProTips", 1000)

NameError: name 'requests' is not defined

### Save JSON Files:

In [4]:
pd.to_pickle(lpt_posts, "../datasets/lpt_posts_json")
pd.to_pickle(ulpt_posts, "../datasets/ulpt_posts_json")

### Read JSON Files:

In [4]:
lpt_posts = pd.read_pickle("../datasets/lpt_posts_json")
ulpt_posts = pd.read_pickle("../datasets/ulpt_posts_json")

# Dataframe

## Create DF

In [11]:
# # Create DF
lpt_df = pd.DataFrame(lpt_posts)
ulpt_df = pd.DataFrame(ulpt_posts)

In [12]:
df = lpt_df.append(ulpt_df, ignore_index=True)

In [7]:
# Read In Existing:
# df = pd.read_pickle("../datasets/df_model_1.2")

In [13]:
df.shape

(2000, 70)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 70 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   all_awardings                  2000 non-null   object 
 1   allow_live_comments            2000 non-null   bool   
 2   author                         2000 non-null   object 
 3   author_flair_css_class         0 non-null      object 
 4   author_flair_richtext          1788 non-null   object 
 5   author_flair_text              0 non-null      object 
 6   author_flair_type              1788 non-null   object 
 7   author_fullname                1788 non-null   object 
 8   author_patreon_flair           1788 non-null   object 
 9   author_premium                 1788 non-null   object 
 10  awarders                       2000 non-null   object 
 11  can_mod_post                   2000 non-null   bool   
 12  contest_mode                   2000 non-null   b

In [15]:
df["subreddit"].head()

0    LifeProTips
1    LifeProTips
2    LifeProTips
3    LifeProTips
4    LifeProTips
Name: subreddit, dtype: object

In [16]:
# print((df["selftext"] == "[removed]").sum())
removed_selftext = (df["selftext"] == "[removed]").sum()
deleted_selftext = (df["selftext"] == "[deleted]").sum()
nan_selftext = (df["selftext"].isna().sum())

print(f"Percent [removed]: {np.round((removed_selftext / len(df)) * 100)}%")
print(f"Percent [deleted]: {np.round((deleted_selftext / len(df)) * 100)}%")
print(f"Percent NaN: {np.round((nan_selftext / len(df)) * 100)}%")

Percent [removed]: 46.0%
Percent [deleted]: 3.0%
Percent NaN: 8.0%


## Exploration

- Investigate additional features: `score`, `author`, `over_18`

    - `author` 
         - Set cvec hyparam 'lowercase' == False
         - Has a number of `[deleted]` author names

- Notes for next model:
    - `created_utc` & `retrieved_on` as features

In [9]:
# Column names
df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'suggested_

# Feature Engineering & Preprocessing

### Cleaning:

- **HTML Artifacts:**
- **Non-Letters**
- **Stopwords**
- **Lemmatize words**

**Lemmatize:**

- This can help with some typos in our word analysis.
    - For example, we can use lemmatization to identify `untill`, and make a necessary adjustment to model input
- Lemmatization will not be applied to `author`, as these are the usernames attached to the post submission to the subreddit.

In [17]:
tokenizer = RegexpTokenizer(r'\w+')
lemma = WordNetLemmatizer()
def to_lemma(data, col):
    for i in range(len(data[col])):
        data[col][i] = tokenizer.tokenize(data[col][i])
        for j in data[col][i]:
            data[col][i] = [lemma.lemmatize(j) for j in data[col][i]]
    data[col] = data[col].apply(lambda i: " ".join(i))
    return data

### Define Stopwords
- In this iteration of the model, the `LPT` or `lpt` word will be removed from the `title` and `selftext` as a stopword.

In [18]:
stopset = set(nltk.corpus.stopwords.words("english"))
stopset.add("lpt")
stopset.add("lptrequest")

stopset.add("ulpt")
stopset.add("ulptrequest")

## Function to perform each Preprocessing task:

- The idea behind this function is to have a function to call on a given feeature that should have all of the preprocessing tasks performed, as listed above.
    - Otherwise, each of the above functions can be called on a feature as needed.

In [19]:
def clean_df(data, col):

    # Remove non-letters:
    new_lst = []
    for i in data[col]:
        soup = BeautifulSoup(i, "lxml")
        new_lst.append(re.sub("[^a-zA-Z]", " ", soup.get_text()))
    data[col] = new_lst
    # Reference to: https://www.reddit.com/r/learnpython/comments/an62wx/how_to_remove_html_from_pandas_dataframe_without/
    
    
    # Make lowercase:
    data[col] = data[col].str.lower()
    

    # Lemmatize:
    for i in range(len(data[col])):
        data[col][i] = tokenizer.tokenize(data[col][i])
        for j in data[col][i]:
            data[col][i] = [lemma.lemmatize(j) for j in data[col][i]]
    data[col] = data[col].apply(lambda i: " ".join(i))
    

    # Remove Stopwords:
    data[col] = [" ".join([i for i in x.split()
                           if i not in stopset])
                           for x in data[col]]
    
    return data

In [20]:
clean_df(df, "title");

/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/bs4/__init__.py:389: UserWarning: "https://www.reddit.com/r/vancouver/comments/g540z0/when_you_see_guide_dogs_make_way_for_them_and/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [21]:
df.shape

(2000, 70)

### Binarize target `y` variable

In [22]:
df["subreddit"].value_counts()

LifeProTips             1000
UnethicalLifeProTips    1000
Name: subreddit, dtype: int64

In [23]:
# Create numeric values for y var to be passed into model

df["subreddit"] = df["subreddit"].map({"LifeProTips": 1,
                                       "UnethicalLifeProTips": 0
                                      })

In [24]:
df["subreddit"].value_counts()

1    1000
0    1000
Name: subreddit, dtype: int64

### Save Dataframe:

In [25]:
pd.to_pickle(df, "../datasets/df_model_1.2")

# Features Set

- Mode word: `videochat`, used 3406 times across all posts

In [5]:
features = df["title"]

In [6]:
X = features
y = df["subreddit"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [73]:
X_train.head()

81      financial situation secure please consider don...
915     want read unbiased major news story regarding ...
1018                   request dealing loud rude neighbor
380     refill empty dish soap bottle dog shampoo easi...
1029              request college exam school make easier
Name: title, dtype: object

### The CountVectorizer DF

In [76]:
cvec_check = CountVectorizer()
cvec_check.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [77]:
X_train = cvec_check.transform(X_train)

In [78]:
X_train_df = pd.DataFrame(X_train.toarray(),
                          columns=cvec_check.get_feature_names())

In [79]:
X_train_df.head()

,abandoned,ability,able,absentee,absolutely,abundance,abusing,ac,academic,accepts,...,younger,youtube,yr,yummy,zaxbys,zero,zip,zipcar,zoom,zoombombed
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
X_train_df["youtube"].value_counts()  # Youtube appears in ULPT 1334 times. 6 times in LPT

0    1334
1       6
Name: youtube, dtype: int64

In [80]:
word_rates = cvec_check.vocabulary_

In [81]:
def get_mode_word(dct):
    for k, v in dct.items():
        highest_k =""
        highest_v = 0
        if v > highest_v:
            highest_v = v
            highest_k = k
    return f"Highest frequency word: {highest_k}, {highest_v} appearances"

In [82]:
get_mode_word(word_rates)

'Highest frequency word: videochat, 3406 appearances'

# Models

### Baseline Score & Train/Test Split

In [8]:
# Read In Existing:
df = pd.read_pickle("../datasets/df_model_1.2")

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [10]:
y_train.head()

81      1
915     1
1018    0
380     1
1029    0
Name: subreddit, dtype: int64

**Baseline Accuracy**

In [11]:
y_test.value_counts(normalize=True)

0    0.507576
1    0.492424
Name: subreddit, dtype: float64

## Logistic Regression | CountVectorizer

In [34]:
logreg_pipe = Pipeline([
    ("cvec", CountVectorizer()),
    ("ss", StandardScaler(with_mean=False)),
    ("logreg", LogisticRegression())
])

**Looking at an untweaked model:**

In [35]:
# Testing the model performance on new data
print(f"CV Score Mean: {cross_val_score(logreg_pipe, X_train, y_train, cv=5).mean()}\n")

# Fit
logreg_pipe.fit(X_train, y_train)

# Train Score
print(f"Train Score: {logreg_pipe.score(X_train, y_train)}")

# Test Score
print(f"Test Score: {logreg_pipe.score(X_test, y_test)}")
logreg_default_score = logreg_pipe.score(X_test, y_test)

CV Score Mean: 0.7925373134328358

Train Score: 0.9970149253731343
Test Score: 0.7863636363636364


### Tuning:

In [47]:
logreg_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     
    "cvec__max_features": [5000],  
    "cvec__min_df": [1],  
    "logreg__penalty": ["l2"],
    "logreg__C": [0.01, 0.05, 0.1],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
    "logreg__verbose": [1]  # Needs any positive int when using 'liblinear' solver
}

logreg_gs = GridSearchCV(logreg_pipe,
                         logreg_params,
                         cv=5,
                         verbose=10)

# consistently: liblinear best, 5000 max features best

In [48]:
logreg_gs.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.01, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.01, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.776, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.01, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.01, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.780, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.01, logreg__penalty=l2, logreg__random_state=42, logreg__solver=li

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s


[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.776, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.776, total=   0.1s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.799, total=   0.1s
[CV]

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s remaining:    0.0s


[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.05, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.821, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.1, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.1, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.776, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.1, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1 
[LibLinear][CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), logreg__C=0.1, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, score=0.791, total=   0.0s
[CV] cve

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    2.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cvec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [49]:
print(f"Train Score: {logreg_gs.score(X_train, y_train)}")
print(f"Test Score: {logreg_gs.score(X_test, y_test)}\n")
print(f"Best Score: {logreg_gs.best_score_}")

Train Score: 0.996268656716418
Test Score: 0.7833333333333333

Best Score: 0.7962686567164179


In [50]:
best_params = logreg_gs.best_params_
best_params

{'cvec__max_features': 5000,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 1),
 'logreg__C': 0.1,
 'logreg__penalty': 'l2',
 'logreg__random_state': 42,
 'logreg__solver': 'liblinear',
 'logreg__verbose': 1}

In [40]:
logreg_df = pd.DataFrame(logreg_gs.cv_results_)
logreg_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_cvec__max_features,param_cvec__min_df,param_cvec__ngram_range,param_logreg__C,param_logreg__penalty,param_logreg__random_state,...,param_logreg__verbose,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.056269,0.007463,0.005860,0.001106,5000,1,"(1, 1)",0.1,l2,42,...,1,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.779851,0.783582,0.783582,0.794776,0.828358,0.794030,0.017879,1
1,0.056612,0.011359,0.004369,0.000234,5000,1,"(1, 1)",0.5,l2,42,...,1,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.768657,0.779851,0.772388,0.805970,0.835821,0.792537,0.025263,4
2,0.058852,0.006363,0.004524,0.000324,5000,1,"(1, 1)",1,l2,42,...,1,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.768657,0.779851,0.772388,0.805970,0.835821,0.792537,0.025263,4
3,0.116499,0.026932,0.007344,0.001190,5000,1,"(1, 2)",0.1,l2,42,...,1,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.787313,0.798507,0.787313,0.750000,0.824627,0.789552,0.024020,15
4,0.106016,0.027983,0.007024,0.000877,5000,1,"(1, 2)",0.5,l2,42,...,1,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.776119,0.794776,0.791045,0.772388,0.813433,0.789552,0.014662,15


### Tuning Iterations:

- **This absolutely needs to be parsed down to a smaller number of models to be fit. This `gs` is 12,960.**
- **Hyperparameters**:        
    ```python
    logreg_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     
    "cvec__max_features": [1000, 2500, 5000],  
    "cvec__min_df": [1, 2, 5, 10],  
    "logreg__penalty": ["l1", "l2"],
    "logreg__C": [1, 5, 10],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["saga", "liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
    "logreg__max_iter": [100, 250, 500],
    "logreg__verbose": [1, 5, 10]  # Needs any positive int when using 'liblinear' solver
    }
    ```
    - **`ConvergenceWarning`:** Need to be wary of this.
- **Next steps:**
    1) Reduce number of models to fit.
---
---
- **Hyperparameters**:        
    ```python
    logreg_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     
    "cvec__max_features": [2500, 5000],  
    "cvec__min_df": [1, 2, 5, 10],  
    "logreg__penalty": ["l1", "l2"],
    "logreg__C": [1, 5, 10],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["saga", "liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
#     "logreg__max_iter": [100, 250, 500],  # This is being taken out, too many models
    "logreg__verbose": [1]  # Needs any positive int when using 'liblinear' solver
}
    ```
    - Score results:
        ```python
        Train Score: 0.9850746268656716
        Test Score: 0.7893939393939394

        Best Score: 0.7835820895522388
        ```
    - Best Parameters Returned:
        ```python
        {'cvec__max_features': 5000,
         'cvec__min_df': 1,
         'cvec__ngram_range': (1, 1),
         'logreg__C': 1,
         'logreg__penalty': 'l2',
         'logreg__random_state': 42,
         'logreg__solver': 'liblinear',
         'logreg__verbose': 1}
        ```
---
---
**Best Score:**
```python
Train Score: 0.996268656716418
Test Score: 0.7833333333333333

Best Score: 0.7962686567164179
```
**Best params:**
```python
{'cvec__max_features': 5000,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 1),
 'logreg__C': 0.1,
 'logreg__penalty': 'l2',
 'logreg__random_state': 42,
 'logreg__solver': 'liblinear',
 'logreg__verbose': 1}
```

### Logreg | CVEC Model Score Notes:


There were marginal gains in tuning the hyperparameters of the model.

Ultimately though, the model has remained very overfit, regardless of the penalty applied.

## Logistic Regression | Tf idf

In [51]:
tfidf_logreg_pipe = Pipeline([
    ("tfidf", TfidfVectorizer()),  # Did not apply StandardScalar, as per resource below
    ("logreg", LogisticRegression())
])

# https://datascience.stackexchange.com/questions/33730/should-i-rescale-tfidf-features

**Looking at an untweaked model:**

In [52]:
# Testing the model performance on new data
print(f"CV Score Mean: {cross_val_score(tfidf_logreg_pipe, X_train, y_train, cv=5).mean()}")

# Fit
tfidf_logreg_pipe.fit(X_train, y_train)

# Train Score
print(f"Train Score: {tfidf_logreg_pipe.score(X_train, y_train)}")

# Test Score
print(f"Test Score: {tfidf_logreg_pipe.score(X_test, y_test)}")
logreg_default_score = tfidf_logreg_pipe.score(X_test, y_test)

CV Score Mean: 0.7843283582089552
Train Score: 0.9462686567164179
Test Score: 0.7878787878787878


### Tuning:

In [59]:
tfidf_logreg_params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],     
    "tfidf__max_features": [2500, 5000, 7500],  
    "tfidf__min_df": [1, 2, 5],  
    "logreg__penalty": ["l1", "l2"],
    "logreg__C": [10, 15],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["saga", "liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
    "logreg__verbose": [1]  # Needs any positive int when using 'liblinear' solver
}

tfidf_logreg_gs = GridSearchCV(tfidf_logreg_pipe,
                               tfidf_logreg_params,
                               cv=5,
                               verbose=10)

# Model 1: 

In [60]:
tfidf_logreg_gs.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] logreg__C=10, logreg__penalty=l1, logreg__random_state=42, logreg__solver=saga, logreg__verbose=1, tfidf__max_features=2500, tfidf__min_df=1, tfidf__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 1 seconds
[CV]  logreg__C=10, logreg__penalty=l1, logreg__random_state=42, logreg__solver=saga, logreg__verbose=1, tfidf__max_features=2500, tfidf__min_df=1, tfidf__ngram_range=(1, 1), score=0.750, total=   0.4s
[CV] logreg__C=10, logreg__penalty=l1, logreg__random_state=42, logreg__solver=saga, logreg__verbose=1, tfidf__max_features=2500, tfidf__min_df=1, tfidf__ngram_range=(1, 1) 


/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 0 seconds
[CV]  logreg__C=10, logreg__penalty=l1, logreg__random_state=42, logreg__solver=saga, logreg__verbose=1, tfidf__max_features=2500, tfidf__min_df=1, tfidf__ngram_range=(1, 1), score=0.701, total=   0.4s
[CV] logreg__C=10, logreg__penalty=l1, logreg__random_state=42, logreg__solver=saga, logreg__verbose=1, tfidf__max_features=2500, tfidf__min_df=1, tfidf__ngram_range=(1, 1) 


...

[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 2), score=0.784, total=   0.1s
[CV] logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 2) 
[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 2), score=0.780, total=   0.1s
[CV] logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 2) 
[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=5000, tfidf__min_df=2, tfidf__ngram_range=(1, 2), score=0.780, to

[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=1, tfidf__ngram_range=(1, 2), score=0.772, total=   0.1s
[CV] logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=1, tfidf__ngram_range=(1, 2) 
[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=1, tfidf__ngram_range=(1, 2), score=0.780, total=   0.1s
[CV] logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=1, tfidf__ngram_range=(1, 2) 
[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=1, tfidf__ngram_range=(1, 2), score=0.795, to

[LibLinear][CV]  logreg__C=15, logreg__penalty=l2, logreg__random_state=42, logreg__solver=liblinear, logreg__verbose=1, tfidf__max_features=7500, tfidf__min_df=5, tfidf__ngram_range=(1, 2), score=0.698, total=   0.1s
[LibLinear]

[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:  2.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [61]:
print(f"Train Score: {tfidf_logreg_gs.score(X_train, y_train)}")
print(f"Test Score: {tfidf_logreg_gs.score(X_test, y_test)}\n")
print(f"Best Score: {tfidf_logreg_gs.best_score_}")

Train Score: 0.9955223880597015
Test Score: 0.7984848484848485

Best Score: 0.7992537313432836


In [62]:
best_params = tfidf_logreg_gs.best_params_
best_params

{'logreg__C': 10,
 'logreg__penalty': 'l2',
 'logreg__random_state': 42,
 'logreg__solver': 'liblinear',
 'logreg__verbose': 1,
 'tfidf__max_features': 5000,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1)}

In [63]:
tfidf_logreg_df = pd.DataFrame(tfidf_logreg_gs.cv_results_)
tfidf_logreg_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logreg__C,param_logreg__penalty,param_logreg__random_state,param_logreg__solver,param_logreg__verbose,param_tfidf__max_features,...,param_tfidf__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.464500,0.064636,0.005970,0.001261,10,l1,42,saga,1,2500,...,"(1, 1)","{'logreg__C': 10, 'logreg__penalty': 'l1', 'lo...",0.750000,0.701493,0.776119,0.772388,0.776119,0.755224,0.028573,82
1,0.557083,0.084224,0.009925,0.003285,10,l1,42,saga,1,2500,...,"(1, 2)","{'logreg__C': 10, 'logreg__penalty': 'l1', 'lo...",0.746269,0.731343,0.761194,0.757463,0.768657,0.752985,0.013012,92
2,0.402278,0.017317,0.006949,0.002143,10,l1,42,saga,1,2500,...,"(1, 1)","{'logreg__C': 10, 'logreg__penalty': 'l1', 'lo...",0.764925,0.720149,0.779851,0.776119,0.768657,0.761940,0.021552,59
3,0.507915,0.031907,0.008356,0.001134,10,l1,42,saga,1,2500,...,"(1, 2)","{'logreg__C': 10, 'logreg__penalty': 'l1', 'lo...",0.761194,0.723881,0.779851,0.768657,0.772388,0.761194,0.019603,65
4,0.287274,0.029836,0.006894,0.002040,10,l1,42,saga,1,2500,...,"(1, 1)","{'logreg__C': 10, 'logreg__penalty': 'l1', 'lo...",0.746269,0.697761,0.723881,0.720149,0.682836,0.714179,0.021961,121


### Tuning Notes:

- **This absolutely needs to be parsed down to a smaller number of models to be fit. This `gs` is 12,960.**
- **Hyperparameters**:        
    ```python
    tfidf_logreg_params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],     
    "tfidf__max_features": [2500, 5000],  
    "tfidf__min_df": [1, 2, 5, 10],  
    "logreg__penalty": ["l1", "l2"],
    "logreg__C": [1, 5, 10],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["saga", "liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
#     "logreg__max_iter": [100, 250, 500],  # This is being taken out, too many models
    "logreg__verbose": [1]  # Needs any positive int when using 'liblinear' solver
    }
    ```
    - Score results:
        ```python
    Train Score: 0.9955223880597015
    Test Score: 0.7984848484848485

    Best Score: 0.7992537313432836
        ```
    - Best Parameters Returned:
        ```python
    {'logreg__C': 10,
     'logreg__penalty': 'l2',
     'logreg__random_state': 42,
     'logreg__solver': 'liblinear',
     'logreg__verbose': 1,
     'tfidf__max_features': 5000,
     'tfidf__min_df': 1,
     'tfidf__ngram_range': (1, 1)}
        ```
---
---
- **Hyperparameters**:        
    ```python
    tfidf_logreg_params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],     
    "tfidf__max_features": [2500, 5000, 7500],  
    "tfidf__min_df": [1, 2, 5],  
    "logreg__penalty": ["l1", "l2"],
    "logreg__C": [10, 15],
    "logreg__random_state": [42],  # Needs a random_state when using the solvers below
    "logreg__solver": ["saga", "liblinear"],  # Will adjust to "lbfgs" if model prefers "l2" penalty
    "logreg__verbose": [1]  # Needs any positive int when using 'liblinear' solver
    }
    ```
    - Score results:
    ```python
    Train Score: 0.9955223880597015
    Test Score: 0.7984848484848485

    Best Score: 0.7992537313432836
    ```
    - Best Parameters Returned:
    ```python
    {'logreg__C': 10,
     'logreg__penalty': 'l2',
     'logreg__random_state': 42,
     'logreg__solver': 'liblinear',
     'logreg__verbose': 1,
     'tfidf__max_features': 5000,
     'tfidf__min_df': 1,
     'tfidf__ngram_range': (1, 1)}
    ```

### Logreg | CVEC Model Score Notes:


Still a lot of overfitting- with the hyparams I've tried so far, it doesn't seem like LogReg is getting any more accurate.

## K-Nearest Neighbors | CountVectorizer

In [64]:
ss = StandardScaler(with_mean=False)  # Error message indicated to add this to the scalar

In [65]:
knn_pipe = Pipeline([
    ("cvec", CountVectorizer()),
    ("ss", ss),
    ("knn", KNeighborsClassifier())
])

**Looking at an untweaked model:**

In [66]:
# Testing the model performance on new data
print(f"CV Score Mean: {cross_val_score(knn_pipe, X_train, y_train, cv=5).mean()}\n")

# Fit
knn_pipe.fit(X_train, y_train)

# Train Score
print(f"Train Score: {knn_pipe.score(X_train, y_train)}")

# Test Score
print(f"Test Score: {knn_pipe.score(X_test, y_test)}")
knn_default_score = knn_pipe.score(X_test, y_test)

CV Score Mean: 0.658955223880597

Train Score: 0.7731343283582089
Test Score: 0.6651515151515152


### Tuning:

**GridSearchCV (gs)**

In [67]:
knn_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     
    "cvec__max_features": [3000, 5000, 7500],  
    "cvec__min_df": [1, 2, 5, 10],             
    "knn__n_neighbors": [1, 3, 5],             
#     "knn__p": [1, 2] 
    "knn__metric": ["euclidian", "manhattan", "minkowski"]
}


gs = GridSearchCV(knn_pipe,
                  knn_params,
                  cv=5,
                  verbose=10)

# 960 Models

In [68]:
gs.fit(X_train, y_train);

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1, score=nan, total=   0.1s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailed

[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=1, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidi

/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.5s remaining:    0.0s
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/

[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3, score=nan, total=   0.1s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=3, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=5 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=5 
[CV]  cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.0s
[CV] cvec__max_features=3000, cvec__min_df=1, cvec__ngram_range=(1, 1), knn__metric=euclidi

...

/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit fail

[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=euclidian, knn__n_neighbors=5, score=nan, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric

/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Metric 'euclidian' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable function.

  FitFailedWarning)
/Users/patrickmccaul/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit fail

[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1, score=0.657, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1, score=0.646, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1, score=0.716, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1 
[CV]  cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn__metric=manhattan, knn__n_neighbors=1, score=0.672, total=   0.1s
[CV] cvec__max_features=7500, cvec__min_df=10, cvec__ngram_range=(1, 2), knn

[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  1.3min finished


In [71]:
print(f"Train Score: {gs.score(X_train, y_train)}")
print(f"Test Score: {gs.score(X_test, y_test)}\n")
print(f"Best Score: {gs.best_score_}")

Train Score: 0.9604477611940299
Test Score: 0.7363636363636363

Best Score: 0.7276119402985074


In [72]:
best_params = gs.best_params_
best_params

{'cvec__max_features': 3000,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 1),
 'knn__metric': 'manhattan',
 'knn__n_neighbors': 3}

In [73]:
knn_df = pd.DataFrame(gs.cv_results_)
knn_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_cvec__max_features,param_cvec__min_df,param_cvec__ngram_range,param_knn__metric,param_knn__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.041881,0.007548,0.000000,0.000000,3000,1,"(1, 1)",euclidian,1,"{'cvec__max_features': 3000, 'cvec__min_df': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,216
1,0.047384,0.010271,0.000000,0.000000,3000,1,"(1, 1)",euclidian,3,"{'cvec__max_features': 3000, 'cvec__min_df': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,151
2,0.023508,0.003010,0.000000,0.000000,3000,1,"(1, 1)",euclidian,5,"{'cvec__max_features': 3000, 'cvec__min_df': 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,167
3,0.028659,0.006441,0.036499,0.009064,3000,1,"(1, 1)",manhattan,1,"{'cvec__max_features': 3000, 'cvec__min_df': 1...",0.701493,0.671642,0.634328,0.649254,0.615672,0.654478,0.029813,117
4,0.043077,0.017460,0.042659,0.011693,3000,1,"(1, 1)",manhattan,3,"{'cvec__max_features': 3000, 'cvec__min_df': 1...",0.697761,0.742537,0.750000,0.712687,0.735075,0.727612,0.019460,1


In [165]:
knn_preds = gs.predict(X_test)
knn_cm = confusion_matrix(y_test, preds)
knn_cm_df = pd.DataFrame(cm, index=["actual_lpt", "actual_ulpt"],
                     columns=["pred_lpt", "pred_ulpt"])
knn_cm_df

,pred_lpt,pred_ulpt
actual_lpt,205,130
actual_ulpt,44,281


### Iteration Notes:

- **Hyperparameters**:        
    ```python
    knn_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     # 2
    "cvec__max_features": [1000, 2500, 5000],  # 3
    "cvec__min_df": [1, 2, 5, 10],             # 4
    "knn__n_neighbors": [3, 5, 9, 11],         # 4
    "knn__p": [1, 2]                           # 2
    }
    ```
    - Score results:
        ```python
        Train Score: 0.9604477611940299
        Test Score: 0.7257575757575757
        Best Score: 0.7335820895522388
        ```
    - Best Parameters Returned:
        ```python
        {'cvec__max_features': 5000,
         'cvec__min_df': 1,
         'cvec__ngram_range': (1, 1),
         'knn__n_neighbors': 3,
         'knn__p': 1}
        ```
        - **Note:** The best distance was `Manhattan` distance
- **Next steps: increase cvec `max_features`, lower `n_neighbors`**
---
---
- **Hyperparameters**:        
    ```python
    knn_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     # 2
    "cvec__max_features": [3000, 5000, 7500],  # 3
    "cvec__min_df": [1, 2, 5, 10],             # 4
    "knn__n_neighbors": [1, 3, 5],         # 4
    "knn__p": [1, 2]                           # 2
    }
    ```
    - Score results:
        ```python
        Train Score: 0.9559701492537314
        Test Score: 0.7439393939393939
        Best Score: 0.7335820895522388
        ```
    - Best Parameters Returned:
        ```python
        {'cvec__max_features': 3000,
         'cvec__min_df': 1,
         'cvec__ngram_range': (1, 1),
         'knn__n_neighbors': 3,
         'knn__p': 1}
        ```
- **Note:** The best distance was `Manhattan` distance
---
---
- **Hyperparameters**:        
    ```python
    knn_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],     
    "cvec__max_features": [3000, 5000, 7500],  
    "cvec__min_df": [1, 2, 5, 10],             
    "knn__n_neighbors": [1, 3, 5],             
#     "knn__p": [1, 2] 
    "knn__metric": ["euclidian", "manhattan", "minkowski"]
    }
    ```
    - Score results:
        ```python
    Train Score: 0.9604477611940299
    Test Score: 0.7363636363636363

    Best Score: 0.7276119402985074
        ```
    - Best Parameters Returned:
        ```python
    {'cvec__max_features': 3000,
     'cvec__min_df': 1,
     'cvec__ngram_range': (1, 1),
     'knn__metric': 'manhattan',
     'knn__n_neighbors': 3}
        ```

### KNN Model Score Notes:

This model is overfit.

Overall, we haven't reached especially high accuracy in these predictions.
 
 **Although the bias has increased (evident in how much worse it has become in prediction, even on the train data), it does seem to be getting closer to score equality with testing on data the model has not seen**

## KNN Regression | Tf idf

In [120]:
ss = StandardScaler(with_mean=False)  # Error message said to add this to the scalar

In [74]:
tfidf_knn_pipe = Pipeline([
    ("tfidf", TfidfVectorizer()), 
#     ("ss", ss),
    ("tfidf_knn", KNeighborsClassifier())
])

# Per doc referenced above, did not apply StandardScalar, as this could conflict with the Tf-idf.
# Also, this was my worst score so far.

**Looking at an untweaked model:**

In [75]:
# Testing the model performance on new data
print(f"CV Score Mean: {cross_val_score(tfidf_knn_pipe, X_train, y_train, cv=5).mean()}")

# Fit
tfidf_knn_pipe.fit(X_train, y_train)

# Train Score
print(f"Train Score: {tfidf_knn_pipe.score(X_train, y_train)}")

# Test Score
print(f"Test Score: {tfidf_knn_pipe.score(X_test, y_test)}")
tfidf_knn_default_score = tfidf_knn_pipe.score(X_test, y_test)

CV Score Mean: 0.5477611940298507
Train Score: 0.6611940298507463
Test Score: 0.5484848484848485


### Tuning:

In [85]:
tfidf_knn_params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],     
    "tfidf__max_features": [1000, 2000,3000],  
    "tfidf__min_df": [5, 7, 10],             
    "tfidf_knn__n_neighbors": [1, 3],       
    "tfidf_knn__p": [1, 2]                     
}


tfidf_knn_gs = GridSearchCV(tfidf_knn_pipe,
                  tfidf_knn_params,
                  cv=5,
                  verbose=10)

In [86]:
tfidf_knn_gs.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 
[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.646, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 
[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.694, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.705, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 
[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.713, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 
[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.701, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=2 
[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=2, score=0.627, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.4s remaining:    0.0s


[CV]  tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=2, score=0.690, total=   0.1s
[CV] tfidf__max_features=1000, tfidf__min_df=5, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=1, tfidf_knn__p=2 


...

[CV]  tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1, score=0.616, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1 
[CV]  tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1, score=0.604, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1 
[CV]  tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1, score=0.646, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1 
[CV]  tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=3, tfidf_knn__p=1, score=0.608, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=5, tfidf__ngram_range=(1

[CV]  tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.668, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2 
[CV]  tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2, score=0.646, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2 
[CV]  tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2, score=0.638, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2 
[CV]  tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=2, score=0.731, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=7, tfidf__ngram_range=(1

[CV]  tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=3, tfidf_knn__p=2, score=0.653, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=3, tfidf_knn__p=2 
[CV]  tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=3, tfidf_knn__p=2, score=0.627, total=   0.0s
[CV] tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=3, tfidf_knn__p=2 
[CV]  tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 1), tfidf_knn__n_neighbors=3, tfidf_knn__p=2, score=0.616, total=   0.0s
[CV] tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=1 
[CV]  tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_range=(1, 2), tfidf_knn__n_neighbors=1, tfidf_knn__p=1, score=0.694, total=   0.1s
[CV] tfidf__max_features=3000, tfidf__min_df=10, tfidf__ngram_

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   32.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [87]:
print(f"Train Score: {tfidf_knn_gs.score(X_train, y_train)}")
print(f"Test Score: {tfidf_knn_gs.score(X_test, y_test)}\n")
print(f"Best Score: {tfidf_knn_gs.best_score_}")

Train Score: 0.9880597014925373
Test Score: 0.6893939393939394

Best Score: 0.7


In [79]:
best_params = tfidf_knn_gs.best_params_
best_params

{'tfidf__max_features': 3000,
 'tfidf__min_df': 5,
 'tfidf__ngram_range': (1, 2),
 'tfidf_knn__n_neighbors': 1,
 'tfidf_knn__p': 2}

In [80]:
tfidf_knn_df = pd.DataFrame(tfidf_knn_gs.cv_results_)
tfidf_knn_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tfidf__max_features,param_tfidf__min_df,param_tfidf__ngram_range,param_tfidf_knn__n_neighbors,param_tfidf_knn__p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.039373,0.007211,0.043171,0.007312,3000,1,"(1, 1)",1,1,"{'tfidf__max_features': 3000, 'tfidf__min_df':...",0.630597,0.611940,0.615672,0.638060,0.608209,0.620896,0.011464,100
1,0.031412,0.011216,0.024221,0.007781,3000,1,"(1, 1)",1,2,"{'tfidf__max_features': 3000, 'tfidf__min_df':...",0.623134,0.623134,0.638060,0.652985,0.649254,0.637313,0.012576,77
2,0.029711,0.004008,0.036448,0.005559,3000,1,"(1, 1)",3,1,"{'tfidf__max_features': 3000, 'tfidf__min_df':...",0.634328,0.656716,0.634328,0.630597,0.649254,0.641045,0.010123,62
3,0.030559,0.007395,0.021594,0.003615,3000,1,"(1, 1)",3,2,"{'tfidf__max_features': 3000, 'tfidf__min_df':...",0.626866,0.675373,0.649254,0.634328,0.690299,0.655224,0.024136,35
4,0.035913,0.007423,0.036311,0.002604,3000,1,"(1, 1)",5,1,"{'tfidf__max_features': 3000, 'tfidf__min_df':...",0.522388,0.537313,0.533582,0.570896,0.514925,0.535821,0.019259,143


### Tuning Notes:

- **With SS Hyperparameters**:                
    ```python
    tfidf_knn_params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],     
    "tfidf__max_features": [3000, 5000, 7500],  
    "tfidf__min_df": [1, 2, 5, 10],             
    "tfidf_knn__n_neighbors": [1, 3, 5],       
    "tfidf_knn__p": [1, 2]                     
    }
    ```
    - Score results:
    ```python
    Train Score: 0.9880597014925373
    Test Score: 0.6893939393939394

    Best Score: 0.7
    ```
    - Best Parameters Returned:
    ```python
    {'tfidf__max_features': 3000,
     'tfidf__min_df': 5,
     'tfidf__ngram_range': (1, 2),
     'tfidf_knn__n_neighbors': 1,
     'tfidf_knn__p': 2}
    ```
    **Best Distance was Euclidian**
---
---
- **W/ No SS Hyperparameters**:        

    - Score results:
    ```python
    Train Score: 0.9880597014925373
    Test Score: 0.6893939393939394

    Best Score: 0.7
    ```
    - Best Parameters Returned:
    ```python
    {'tfidf__max_features': 3000,
     'tfidf__min_df': 5,
     'tfidf__ngram_range': (1, 2),
     'tfidf_knn__n_neighbors': 1,
     'tfidf_knn__p': 2}
    ```

### KNN Tf-idf Model Score Notes:


I tried this model both with and without a StandardScalar applied to the pipeline. This did not seem to make much of any difference in the accuracy scores.

Slightly less overfit than the logistic regression models, but still overfit

## Multinomial Naive Bayes Model | CountVectorizer

In [166]:
nb_pipe = Pipeline([
    ("cvec", CountVectorizer()),
    ("ss", StandardScaler(with_mean=False)),
    ("nb", MultinomialNB())
])

**Looking at an untweaked model:**

In [167]:
# Testing the model performance on new data
print(f"CV Score Mean: {cross_val_score(nb_pipe, X_train, y_train, cv=5).mean()}")

# Fit
nb_pipe.fit(X_train, y_train)

# Train Score
print(f"Train Score: {nb_pipe.score(X_train, y_train)}")

# Test Score
print(f"Test Score: {nb_pipe.score(X_test, y_test)}")
nb_default_score = nb_pipe.score(X_test, y_test)

CV Score Mean: 0.7492537313432837
Train Score: 0.9850746268656716
Test Score: 0.7787878787878788


### Tuning:

In [168]:
nb_params = {
    "cvec__ngram_range": [(1, 1), (1, 2)],
    "cvec__min_df": [1, 2],                  
    "cvec__max_features": [5000, 7500, 10_000],
    "nb__alpha": [2.0, 3.0, 4.0],
    "nb__fit_prior": [True, False]
}

nb_gs = GridSearchCV(nb_pipe,
                  nb_params,
                  cv=5,
                  verbose=10)

# 540 Models

In [170]:
nb_gs.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True, score=0.728, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True, score=0.728, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True, score=0.761, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=Tru

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s


[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False, score=0.731, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False, score=0.765, total=   0.1s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False, score=0.750, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False, score=0.780, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.4s remaining:    0.0s


[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.757, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.761, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.780, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=False 
[CV]  cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=False, score=0.720, total=   0.0s
[CV] cvec__max_features=5000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=False 
[CV]  cvec__max_features=5000

...

[CV]  cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 1), nb__alpha=4.0, nb__fit_prior=False, score=0.784, total=   0.0s
[CV] cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True, score=0.750, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True, score=0.765, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True, score=0.784, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=1, cvec__ngram_range=(1, 2), nb__alpha=2.0, nb__fit_prior=True 
[CV]  cvec__max_feature

[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=2.0, nb__fit_prior=False, score=0.746, total=   0.0s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.731, total=   0.0s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.728, total=   0.0s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True, score=0.724, total=   0.0s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 1), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_feature

[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=True, score=0.720, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=True 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=True, score=0.772, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=False 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=False, score=0.746, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=False 
[CV]  cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=False, score=0.746, total=   0.1s
[CV] cvec__max_features=10000, cvec__min_df=2, cvec__ngram_range=(1, 2), nb__alpha=3.0, nb__fit_prior=False 
[CV]  cvec__max_fea

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:   19.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cvec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [171]:
print(f"Train Score: {nb_gs.score(X_train, y_train)}")
print(f"Test Score: {nb_gs.score(X_test, y_test)}\n")
print(f"Best Score: {nb_gs.best_score_}")

Train Score: 0.9940298507462687
Test Score: 0.8106060606060606

Best Score: 0.7776119402985074


In [173]:
best_params = nb_gs.best_params_
best_params

{'cvec__max_features': 10000,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 2),
 'nb__alpha': 4.0,
 'nb__fit_prior': True}

In [174]:
nb_df = pd.DataFrame(nb_gs.cv_results_)
nb_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_cvec__max_features,param_cvec__min_df,param_cvec__ngram_range,param_nb__alpha,param_nb__fit_prior,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.031312,0.008320,0.005128,0.000829,5000,1,"(1, 1)",2,True,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.727612,0.727612,0.761194,0.761194,0.783582,0.752239,0.021706,17
1,0.027060,0.005171,0.011283,0.009531,5000,1,"(1, 1)",2,False,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.720149,0.731343,0.764925,0.750000,0.779851,0.749254,0.021680,27
2,0.027860,0.007621,0.006789,0.004226,5000,1,"(1, 1)",3,True,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.727612,0.727612,0.757463,0.761194,0.779851,0.750746,0.020355,24
3,0.028284,0.007278,0.004589,0.000711,5000,1,"(1, 1)",3,False,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.720149,0.731343,0.761194,0.750000,0.776119,0.747761,0.020108,38
4,0.022875,0.005410,0.004340,0.000365,5000,1,"(1, 1)",4,True,"{'cvec__max_features': 5000, 'cvec__min_df': 1...",0.731343,0.727612,0.757463,0.768657,0.787313,0.754478,0.022562,13


In [175]:
nb_preds = nb_gs.predict(X_test)
nb_cm = confusion_matrix(y_test, nb_preds)
nb_cm_df = pd.DataFrame(nb_cm, index=["actual_lpt", "actual_ulpt"],
                     columns=["pred_lpt", "pred_ulpt"])
nb_cm_df

,pred_lpt,pred_ulpt
actual_lpt,285,50
actual_ulpt,75,250


### Tuning Notes:

- **Hyperparameters**:        
    ```python
    nb_params = {
    "cvec__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "cvec__min_df": [1, 2, 5, 10],                  
    "cvec__max_features": [2500, 5000, 10_000],
    "nb__alpha": [0, 1.0, 2.0, 3.0],
    "nb__fit_prior": [True, False]
    ```
    - Score results:
        ```python
        Train Score: 0.9738805970149254
        Test Score: 0.7984848484848485
        Best Score: 0.7977611940298507
        ```
    - Best Parameters Returned:
        ```python
        {'cvec__max_features': 10000,
         'cvec__min_df': 1,
         'cvec__ngram_range': (1, 2),
         'nb__alpha': 2.0,
         'nb__fit_prior': True}
        ```
    - **Notes:** These scores are less overfit. Not super accurate as a whole, but a huge improvement over KNN. The model is much more accurate on test data
- **Next steps:** Increase the `max_features`, and increase `ngram_range`

- **Hyperparameters**:        
    ```python
    "cvec__ngram_range": [(1, 2), (1, 3), (1, 4)],
    "cvec__min_df": [1, 2],                  
    "cvec__max_features": [10_000, 12_500, 15_000],
    "nb__alpha": [1.0, 2.0, 3.0],
    "nb__fit_prior": [True, False]
    ```
    - Score results:
        ```python
        Train Score: 0.9798507462686568
        Test Score: 0.7954545454545454
        Best Score: 0.8044776119402985
        ```
    - Best Parameters Returned:
        ```python
        {'cvec__max_features': 12500,
         'cvec__min_df': 1,
         'cvec__ngram_range': (1, 2),
         'nb__alpha': 2.0,
         'nb__fit_prior': True}
        ```
- **Notes:** Has a better `best_score_`, and the best score so far.
    - Score results:
    ```python
    Train Score: 0.9940298507462687
    Test Score: 0.8106060606060606

    Best Score: 0.7776119402985074
    ```
    - Best Parameters Returned:
    ```python
    {'cvec__max_features': 10000,
     'cvec__min_df': 1,
     'cvec__ngram_range': (1, 2),
     'nb__alpha': 4.0,
     'nb__fit_prior': True}
    ```


### Multinomial Naive Bayes Model Score Notes:


This model is, again, overfit. However, it did return the most accurate score to date, in its second iteration!